In [135]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, max,  col, round

In [136]:
# Criação da SparkSession
spark = SparkSession.builder.appName('Hello World!').getOrCreate()

# Leitura do dataset
df = spark.read.csv('/content/gas_prices.csv', header=True, inferSchema=True)

In [166]:
# Exibir amostra e schema
df.show(5)
df.printSchema()

+----+---------+------+------+-------+-----+-----+------+-----------+----+----+
|Year|Australia|Canada|France|Germany|Italy|Japan|Mexico|South Korea|  UK| USA|
+----+---------+------+------+-------+-----+-----+------+-----------+----+----+
|1990|     NULL|  1.87|  3.63|   2.65| 4.59| 3.16|   1.0|       2.05|2.82|1.16|
|1991|     1.96|  1.92|  3.45|    2.9|  4.5| 3.46|   1.3|       2.49|3.01|1.14|
|1992|     1.89|  1.73|  3.56|   3.27| 4.53| 3.58|   1.5|       2.65|3.06|1.13|
|1993|     1.73|  1.57|  3.41|   3.07| 3.68| 4.16|  1.56|       2.88|2.84|1.11|
|1994|     1.84|  1.45|  3.59|   3.52|  3.7| 4.36|  1.48|       2.87|2.99|1.11|
+----+---------+------+------+-------+-----+-----+------+-----------+----+----+
only showing top 5 rows

root
 |-- Year: integer (nullable = true)
 |-- Australia: double (nullable = true)
 |-- Canada: double (nullable = true)
 |-- France: double (nullable = true)
 |-- Germany: double (nullable = true)
 |-- Italy: double (nullable = true)
 |-- Japan: double (

root
 |-- Year: integer (nullable = true)
 |-- Australia: double (nullable = true)
 |-- Canada: double (nullable = true)
 |-- France: double (nullable = true)
 |-- Germany: double (nullable = true)
 |-- Italy: double (nullable = true)
 |-- Japan: double (nullable = true)
 |-- Mexico: double (nullable = true)
 |-- South Korea: double (nullable = true)
 |-- UK: double (nullable = true)
 |-- USA: double (nullable = true)



In [165]:
# Colunas de países (excluindo "Year")
colunas_paises = [col for col in df.columns if col != 'Year']
colunas_paises

['Australia',
 'Canada',
 'France',
 'Germany',
 'Italy',
 'Japan',
 'Mexico',
 'South Korea',
 'UK',
 'USA']

In [156]:
# Preço médio da gasolina nos EUA na década de 90
df_usa = df.select('Year', 'USA')

df_usa_90s = df_usa.filter((col('Year') >= 1990) & (col('Year') <= 1999))

media = df_usa_90s.agg(mean('USA').alias('media USA decada de 90'))

media.show()

+----------------------+
|media USA decada de 90|
+----------------------+
|    1.1490000000000002|
+----------------------+



In [157]:
# Ano com o maior preço no Japão
df_japao = df.select('Year', 'Japan')
max_japao = df_japao.agg(max('Japan').alias('Maior preço do Japão')).collect()[0]['Maior preço do Japão']
maior_valor_japao = df_japao.filter(df_japao.Japan == max_japao)
maior_valor_japao.show()

+----+-----+
|Year|Japan|
+----+-----+
|2008| 5.74|
+----+-----+



In [154]:
# Média global por ano

media_por_ano = sum([col(c) for c in colunas_paises.columns]) / len(colunas_paises.columns)

df_com_media = df.withColumn('Media_global', round(media_por_ano, 2))

df_com_media.select('Year', 'Media_global').show(10)

+----+------------+
|Year|Media_global|
+----+------------+
|1990|        NULL|
|1991|        2.61|
|1992|        2.69|
|1993|         2.6|
|1994|        2.69|
|1995|        2.85|
|1996|        2.91|
|1997|        2.84|
|1998|        2.65|
|1999|        2.87|
+----+------------+
only showing top 10 rows



In [162]:
# Ano com maior média global
valor_max_media = df_com_media.agg(max('Media_global').alias('Maior media global')).collect()[0]['Maior media global']

ano_maior_media = df_com_media.filter(col("Media_global") == valor_max_media)

ano_maior_media.select('Year', 'Media_global').show()

+----+------------+
|Year|Media_global|
+----+------------+
|2008|        5.61|
+----+------------+

